In [3]:
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
from fuzzywuzzy import process
import pickle

import warnings 
warnings.filterwarnings("ignore")

c:\Users\shiva\.conda\envs\hello\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [4]:
movies='movies.csv'
ratings='ratings.csv'

df_movies=pd.read_csv(movies, usecols=['movieId','title'], dtype={'movieId':'int64','title':'str'})
df_ratings=pd.read_csv(ratings, usecols=['userId','movieId','rating'],dtype={'userId':'int64','movieId':'int64','rating':'float64'})

In [5]:
df_movies.iloc[1058]

movieId                                          1375
title      Star Trek III: The Search for Spock (1984)
Name: 1058, dtype: object

In [6]:
df_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9742 non-null   int64 
 1   title    9742 non-null   object
dtypes: int64(1), object(1)
memory usage: 152.3+ KB


In [7]:
pickle.dump(df_movies, open('df_movies.pkl', 'wb'))

In [8]:
df_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   userId   100836 non-null  int64  
 1   movieId  100836 non-null  int64  
 2   rating   100836 non-null  float64
dtypes: float64(1), int64(2)
memory usage: 2.3 MB


In [9]:
df_ratings.shape

(100836, 3)

In [10]:
df_ratings.isnull().sum()

userId     0
movieId    0
rating     0
dtype: int64

In [11]:
movies_users=df_ratings.pivot(index='movieId', columns='userId',values='rating').fillna(0)
mat_movies_users=csr_matrix(movies_users.values)

In [12]:
pickle.dump(mat_movies_users, open('mat_movies_users.pkl', 'wb'))

In [13]:
model_knn= NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=20)

In [14]:
pickle.dump(model_knn, open('model_knn.pkl', 'wb'))

In [15]:
model_knn.fit(mat_movies_users)

NearestNeighbors(algorithm='brute', metric='cosine', n_neighbors=20)

In [16]:
# Recommender(movie_name) => List of Movies recommended

def recommender(movie_name, data,model, n_recommendations ):
    model.fit(data)
    idx=process.extractOne(movie_name, df_movies['title'])[2]
    print('Movie Selected:',df_movies['title'][idx])
    print('Searching for recommendations.....')
    distances, indices=model.kneighbors(data[idx], n_neighbors=n_recommendations)
    for i in indices:
        print(df_movies['title'][i].where(i!=idx))    

In [17]:
# Recommender(movie_name) => List of Movies recommended

def recommender(movie_name, data,model, n_recommendations ):
    model.fit(data)
    idx=process.extractOne(movie_name, df_movies['title'])[2]
    print('Movie Selected:',df_movies['title'][idx])
    print('Searching for recommendations.....')
    distances, indices=model.kneighbors(data[idx], n_neighbors=n_recommendations)
    for i in indices:
        print(df_movies['title'][i].where(i!=idx))

In [18]:
#recommender('Superman', mat_movies_users, model_knn,10)

In [19]:
model_knn.fit(mat_movies_users)
idx=process.extractOne("Superman", df_movies['title'])[2]
print('Movie Selected:',df_movies['title'][idx])
distances, indices=model_knn.kneighbors(mat_movies_users[idx], n_neighbors=5)
#x=[df_movies['title'][i].where(i!=idx) for i in indices]
for i in indices:
        print(df_movies['title'][i].where(i!=idx))

Movie Selected: Superman (1978)
1986                                                  NaN
1985                                Mommie Dearest (1981)
1057               Star Trek II: The Wrath of Khan (1982)
2246                             Ipcress File, The (1965)
2314    Women on the Verge of a Nervous Breakdown (Muj...
Name: title, dtype: object


In [20]:
model_knn.fit(mat_movies_users)
idx=process.extractOne("Superman", df_movies['title'])[2]
distances, indices=model_knn.kneighbors(mat_movies_users[idx], n_neighbors=6)
#x=[df_movies['title'][i].where(i!=idx) for i in indices]
x=[]
for i in indices:
    x.append(df_movies['title'][i].where(i!=idx))

In [21]:
df=pd.DataFrame(x)
y=[]
for i,j in df.iloc[:,:].items():
    y.append(j)
final=[i[0] for i in y]
final=final[1:]

In [22]:
final

['Mommie Dearest (1981)',
 'Star Trek II: The Wrath of Khan (1982)',
 'Ipcress File, The (1965)',
 'Women on the Verge of a Nervous Breakdown (Mujeres al borde de un ataque de nervios) (1988)',
 "Logan's Run (1976)"]

# app.py

In [23]:
model_knn.fit(mat_movies_users)
idx=process.extractOne("Superman", df_movies['title'])[2]
distances, indices=model_knn.kneighbors(mat_movies_users[idx], n_neighbors=6)
#x=[df_movies['title'][i].where(i!=idx) for i in indices]
x=[]
for i in indices:
    x.append(df_movies['title'][i].where(i!=idx))
df=pd.DataFrame(x)
y=[]
for i,j in df.iloc[:,:].items():
    y.append(j)
final=[i[0] for i in y]
final=final[1:]

In [24]:
final

['Mommie Dearest (1981)',
 'Star Trek II: The Wrath of Khan (1982)',
 'Ipcress File, The (1965)',
 'Women on the Verge of a Nervous Breakdown (Mujeres al borde de un ataque de nervios) (1988)',
 "Logan's Run (1976)"]